# Natural Language Processing (NLP) Model
A notebook that predicts loan purspose based on loan description given by the borrower

In [11]:
#Imports

#Data
import pandas as pd
import numpy as np


#Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#Text
import string
from nltk.corpus import stopwords

#ML

#Other
import pickle

#General
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

#NLP - Related
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import  TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [6]:
#Getting the csv file's name with the wrangled data
df=pd.read_csv('data/xyz_corp_lending_data.csv',sep='\t')

#Including only the non-na data for desc and getting only the desc and purpose columns
df=df[df['desc'].isna()==False][['desc','purpose']]

#Encoding the purpose values
#df[['purpose']] = df[['purpose']].astype('category').apply(lambda x: x.cat.codes)

#Using only the 2 highest purpose values
#df=df[(df['purpose']==2)|(df['purpose']==1)]
#df['purpose']=df['purpose'].replace(2,0)

#Improving the desc values. Commented as did not make any difference
#df['desc']=df['desc'].apply(lambda x: x.split('>')[1][:-4] if x[-4:]=='<br>' else x)

C:\Users\mmoy\AppData\Local\Temp\ipykernel_1180\2564360736.py:2: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('data/xyz_corp_lending_data.csv',sep='\t')


In [7]:
#A function that takes in a string of text, then remove all punctuation, removes all stopwords, returns a list of the cleaned text
def text_process(mess):

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords (very common words)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [13]:
#NLP model predicting loan purpose based on loan description given by the borrower.
#The model's input is the method used for NLP. If set to initial it just uses count_vectorizer
#if set to anything else it also uses a TfidfTransformer. It is observed that the model runs better
#with the 'initial' input.

def get_model_nlp(method='initial'):


    #Using the text processing function
    #df['desc']=df['desc'].apply(text_process)

    #Creating X and y variables for input and output
    X=df['desc']
    y=df['purpose']

    if method=='initial':
        #Vectorizing the data
        cv = CountVectorizer()
        cv.fit(X)
        #pickle.dump(cv,open('vector_nlp','wb'))
        X=cv.trasform(X)

        #Splitting the data into train and test
        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

        #Using the Naive-Base Binomial algorith for prediction
        model_nlp = MultinomialNB()
        model_nlp.fit(X_train,y_train)
        predictions = model_nlp.predict(X_test)
    else:
        #Using a Pipeline including a TfidfTransformer
        model_nlp = Pipeline([
        ('bow', CountVectorizer(analyzer=text_process)),
        ('tfidf', TfidfTransformer()),
        ('classifier', MultinomialNB()),]) 

        #Splitting the data into train and test
        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

        #Using the Naive-Base Binomial algorith for prediction
        model_nlp.fit(X_train,y_train)
        predictions = model_nlp.predict(X_test) 
    
    #Printing useful metrics
    #print(confusion_matrix(y_test,predictions))
    print(classification_report(y_test,predictions))
    return model_nlp

#Running the function 
model_nlp=get_model_nlp(method='initial')
#pickle.dump(model_nlp,open('model_nlp','wb'))

c:\Users\mmoy\Documents\XYZ Corp\xyz_corp_venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                    precision    recall  f1-score   support

               car       0.69      0.19      0.30       588
       credit_card       0.52      0.33      0.40      7633
debt_consolidation       0.70      0.86      0.77     21097
       educational       1.00      0.01      0.02        81
  home_improvement       0.68      0.79      0.73      2125
             house       0.00      0.00      0.00       264
    major_purchase       0.51      0.18      0.26       927
           medical       0.70      0.02      0.05       290
            moving       0.80      0.02      0.03       263
             other       0.23      0.22      0.22      1939
  renewable_energy       0.00      0.00      0.00        38
    small_business       0.65      0.68      0.67       780
          vacation       1.00      0.01      0.01       168
           wedding       0.81      0.15      0.26       351

          accuracy                           0.65     36544
         macro avg       0.59      0.2

c:\Users\mmoy\Documents\XYZ Corp\xyz_corp_venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mmoy\Documents\XYZ Corp\xyz_corp_venv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
